## Top

In [1]:
import pandas as pd
import sys,os,inspect
import calendar
from datetime import datetime
from datetime import timedelta
from cronsim import CronSim
import argparse
import json
import PySimpleGUI as sg
#import PySimpleGUIWeb as sg
import pathlib
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import textwrap
import re
from shlex import split
import webbrowser
import subprocess
import collections
import pyglet,tkinter
from pyglet import font

import requests
import gspread
from oauth2client.service_account import ServiceAccountCredentials

global datasets
# import OpenGL
# from OpenGL import GLU
font.add_file('/etc/fonts/fonts/CENTAUR.TTF')
font='Courier 10 bold '
bicHome = "/home/joe/bic_etl/"
numWindows=0
headerStore = {}

colorPairs = [["#D6EAF8","#85C1E9"],["#b3f0ff","#33d6ff"],["#D5F5E3","#A3E4D7"],["#FCF3CF","#F7DC6F"]]
windowsOpen = {}
windowsOpen["main"] = []
windowsOpen["unique"] = []


##########################################################

def compare2Data(df,xrefs):
    global dcols,pcols,rowColors,values,dnotpLines
    dcols = list(df.columns)
    pcols = list(xrefs.keys())
    
    wid = windowsStore["columnAnalysis"]
    rowColors = wid["-TABLECOLUMN-"].metadata[1]
    values = wid["-TABLECOLUMN-"].metadata[0]
   
    colr1 = rowColors[0][1]
    colr2 = rowColors[1][1]
    
    pnotd = []
    dnotp = []
    dnotpLines = {}

    for col in pcols:
        if col not in dcols:
            pnotd.append(col)

    for col in dcols:
        if col not in pcols:
            dnotp.append(col)
            for line in transformProgram:
                if line.find(col) > -1:
                   if col in dnotpLines:
                     dnotpLines[col].append(line)
                   else:
                     dnotpLines[col] = []
                     dnotpLines[col].append(line)
                                 
    for nn,value in enumerate(values):    
        if nn%2 == 0:
            rowColor= colr1
        else:
            rowColor= colr2

        col = value[0]

        if col in pnotd:
            rowColor = "pink"

        colr = (nn,rowColor)
        rowColors.append(colr)

        
    sp = "\n".join(pnotd)
    sd = "\n".join(dnotp)
    
        
    wid["-TABLECOLUMN-"].update(row_colors=rowColors)  
    wid["-PNOTD-"].update(pnotd)    
    wid["-DNOTP-"].update(dnotp)    
    
    
##########################################################

def compareWindow(stats1,df1,file1,title="Compare",colorTheme="'Dark Green 5'"):
    global color1,color2
    header_list = ["Column","% Missing","Missing","string","integer","float","boolean"]
  
    col_widths = [8]*len(header_list)
    col_widths[0] = 25
    columnSortStateTable1 = {}
  

    ## set up sort state for the column in both tables
    for col in header_list:
         columnSortStateTable1[col] = -1
       
            
    valsFile1 = getValues(stats1,header_list)
   
    
    rowFile1Colors = setRowColors(valsFile1,color1,color2,"pink",header_list)
    
    layCol1 = [[sg.Text(f"File 1 {file1}",font="CENTAUR 15")],[sg.Text(f"File 1 Shape {df1.shape}",font="CENTAUR 15")],
               [sg.Table(values=valsFile1,text_color="black", auto_size_columns=False,enable_events=True,num_rows=20,col_widths=col_widths,font="CENTAUR 10",
                   justification='center',pad=(5,5),vertical_scroll_only=False,
                   key='-TABLEFILE-',row_colors=rowFile1Colors,headings = header_list,metadata=columnSortStateTable1)]
               ]
    

    layout = [[sg.Button("Quit")],
    layCol1]  
              
    sg.theme(colorTheme)     
    window2 = sg.Window(title,layout,finalize=True,resizable=True,metadata=[colorTheme,df1])
    a = window2.CurrentLocation()
    screen_width, screen_height = window2.get_screen_dimensions()
    win_width, win_height = window2.size
    x, y = (screen_width - win_width)//2, (screen_height - win_height)//2
    x=200
    y=200
    window2.move(x, y)
    tableFile1 = window2['-TABLEFILE-']
    tableFile1.bind('<Button-1>', "Click")
    headerStore[tableFile1] = header_list
    statsStore[tableFile1] = stats1
    print("STATS ",stats1)
    return window2,tableFile1,valsFile1,rowFile1Colors

#############################################

def dfAnalyze(df):
    stats = {}
    
    for col in df.columns:
        typs = df[col].apply(type).value_counts().to_dict()
        stats[col] = {}
        if str in typs:
           stats[col]["string"] = typs[str]
        else:
           stats[col]["string"] = 0

        if int in typs:
           stats[col]["integer"] = typs[int]
        else:
           stats[col]["integer"] = 0

        if float in typs:
           stats[col]["float"] = typs[float]
        else:
           stats[col]["float"] = 0

        if bool in typs:
           stats[col]["boolean"] = typs[bool]
        else:
           stats[col]["boolean"] = 0

        stats[col]["Missing"] = df[col].isna().sum()
        stats[col]["% Missing"] = round(df[col].isna().sum()/df.shape[0]*100,1)
        
    return stats

############################################# 

def exceptionLog(exception,funCall):
  exception_message = str(exception)
  exception_type, exception_object, exception_traceback = sys.exc_info()
  filename = os.path.split(exception_traceback.tb_frame.f_code.co_filename)[1]
  print(f"{exception_message} {exception_type} {funCall}, Line {exception_traceback.tb_lineno}")

###############################################################

def fullDataColumnMap(cim4x4):
    '''Map the data fields found in the source data file to all the other sources  '''
    global missed
    if cim4x4 in fields:
        invc = fields[cim4x4]["source"].copy()
    else:
        invc = []
    tfc = transformColumns.copy()
    exc = extractColumns.copy()
    excMissed = []
    tfcsij = transformSijColumns.copy()
    loadsij = loadSijColumns.copy()
    cim = cimColumns.copy()
    xrfo2t = xrefsO2T.copy()
    names = []
    columnDict = {}
    names.append("BIC Inventory")
    names.append("Transform Program")

    names.append("Transform - Data")
    names.append("CIM - Data")
    names.append("Transform - SIJ")
    names.append("Load - SIJ")

    for k in exc:
        print("MAP ",k)
    #for k,v in xrefsO2T.items():
        columnDict[k] = []
        nhit=0
        v=""
        if k in invc:
            v=k
            vv=k
            invc.remove(k)
            nhit+=1
        else:
            vv=""
        columnDict[k].append(vv)
        
        if k in xrfo2t:
            v = xrfo2t[k]
            vv=v
            del xrfo2t[k]
            nhit+=1
        else:
            vv = ""
        columnDict[k].append(vv)  

   
        if v in tfc:
           tfc.remove(v)
           cc = v
           nhit+=1
        else:
            cc=""
        columnDict[k].append(cc)


        if v in cim:
           cim.remove(v)
           cc = v
           nhit+=1            
        else:
            cc=""
        columnDict[k].append(cc)



        if v.lower() in tfcsij:
           tfcsij.remove(v.lower())
           cc = v.lower()
           nhit+=1            
        else:
            cc=""
        columnDict[k].append(cc)

        if v.lower() in loadsij:
           loadsij.remove(v.lower())
           cc = v.lower()
           nhit+=1
        else:
            cc=""
        columnDict[k].append(cc)
        
        if nhit == 0:
            excMissed.append(k)
        

    a = pd.DataFrame(columnDict).T
    a.columns = names
    a.reset_index(inplace=True)
    values = a.values.tolist()
    cols = list(a.columns)
   
    missed = {}
    missed["Extract - Data"] = excMissed
    missed["BIC Inventory"] = invc
    missed["Transform Program"] = list(xrfo2t.keys())
    missed["Transform - Data"] = tfc
    missed["CIM - Data"] = cim
    missed["Transform - SIJ"] = tfcsij
    missed["Load - SIJ"] = loadsij
    
    
 #   missed = [invc,list(xrfo2t.keys()),tfc,cim,tfcsij,loadsij]
    return cols,values,missed
        

###############################################################
def getFilesClicked(file,window):
#         if len(values["-FILE1-"]) > 0:
#             file = values["-FILE1-"]
#         elif len(values["-WEB1-"]) > 0:
#             file = values["-WEB1-"]
      
        df = getFile("Local",file,window)
        stats = dfAnalyze(df)

        return df,stats

####################################################################
       
def getFile(how,file,WindowP):
    
    if how == "Local":
        if file[-3:].lower() == "tsv":
            delim = "\t"
            df=pd.read_csv(file,encoding="latin",delimiter=delim)
        elif file[-4:].lower() == "xlsx":
            df=pd.read_excel(file,engine="openpyxl")
        else:
            try:
                df=pd.read_csv(file)
            except Exception as err:
                print("Error, trying with encoding=latin")
                df=pd.read_csv(file,encoding="latin")
    elif how == "Fetch":
      
  #      file=values["-WEB-"]
      #  getPrevFiles(2,file)

  #      windowP["-PINFO-"].update(f"START reading WEB File:{file}:")
        df=pd.read_csv(file)
      
  #      windowP["-PINFO-"].update(f"FINISHED reading WEB File:{file}:")
   
    stats = dfAnalyze(df)
        
    return df,stats

############################################################

def getRowClicked(table,columns):
    col=""
    e = table.user_bind_event 
    region = table.Widget.identify('region', e.x, e.y)
    if region == 'heading':
        row = 0
    elif region == 'cell':
        row = int(table.Widget.identify_row(e.y))  
        col = columns[row-1][0]
        
    return row,col    

#####################################################################

def getRowClickedUN(table):
    val=""
    data = dataStore[table]
    e = table.user_bind_event 
    region = table.Widget.identify('region', e.x, e.y)
    if region == 'heading':
        row = 0
    elif region == 'cell':
        row = int(table.Widget.identify_row(e.y))  
        val = data[row-1][0]
        print("Un VAL CLICKED ",val)
    return row,val    

##########################################################

def getSijColumns(file):
    '''Get the fields for both teh contrile file content and the ftp control file content '''
    global transformSijColumns,loadSijColumns
    with open("/home/joe/bic_etl/cdos/business/nonprofit/scripts/datasync/char_paid_solicitors.sij") as fin:
        lines = fin.readlines()
        line = lines[0]
        ss = json.loads(line)
        cfc = json.loads(ss["controlFileContent"])
        transformSijColumns = cfc["csv"]["columns"]
        ftp = json.loads(ss["ftpControlFileContent"])
        loadSijColumns = ftp['csv']['columns']

#         for col in transformColumns:
#             if col in loadColumns:
#                 hit+=1
#             else:
#                 miss+=1

#         for col in loadColumns:
#             if col in transformColumns:
#                 hit+=1
#             else:
#                 miss+=1

# ##########################################################

def getValues(stats,header):

    statsVals=[]
    for col in sorted(stats.keys()):
         vals=[]
         vals.append(col)
         for k in header[1:]:
            vals.append(stats[col][k.strip()])
         statsVals.append(vals)
    return statsVals     

##############################################################

def getXrefs():
    '''Reads the Inventory google sheet and gets the datasets title and cross-refs it to the Socrata 4x4 id.  Also
    gets the fields by 4x4 dataset id and by the title'''
    scope = ['https://www.googleapis.com/auth/spreadsheets.readonly',
             "https://www.googleapis.com/auth/drive.file",
                  "https://www.googleapis.com/auth/drive"]

    creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json',
     scope)
    client = gspread.authorize(creds)

    gc = gspread.service_account("./client_secret.json")
    for gg in gc.list_spreadsheet_files():
         print("GGGGG ",gg)
    # https://docs.google.com/spreadsheets/d/1WTaOglzbSsYiHhAGguGxHQXmAGmOhfFHkGkMLowxAOA/edit?usp=sharing
    sheet = client.open('BIC Data Inventory and Metadata').worksheet(
        'Maintenance_Framework')
    repo_sheet = client.open('BIC Data Inventory and Metadata').worksheet(
        'MetadataRepository')
    fields_sheet = client.open('BIC Data Inventory and Metadata').worksheet(
        'Field Descriptions')
    
    
#     sheet = client.open('https://docs.google.com/spreadsheets/d/1Xc7oYpdCLwHmUCaokpfXkF4bzkwRW0xUbvBZwruF0ZI/').worksheet(
#         'Maintenance_Framework')
#     repo_sheet = client.open('https://docs.google.com/spreadsheets/d/1Xc7oYpdCLwHmUCaokpfXkF4bzkwRW0xUbvBZwruF0ZI/').worksheet(
#         'MetadataRepository')
    
#     fields_sheet = client.open('https://docs.google.com/spreadsheets/d/1Xc7oYpdCLwHmUCaokpfXkF4bzkwRW0xUbvBZwruF0ZI/').worksheet(
#         'Field Descriptions')

    dfRepo = pd.DataFrame(repo_sheet.get_all_records(head=3))
    xrefsBy4x4 = {}
    xrefsByTitle = {}

    for index,row in dfRepo[['Dataset Title','Socrata Link']].iterrows():
        xrefsByTitle[row['Dataset Title']] = row['Socrata Link']
        xrefsBy4x4[row['Socrata Link']] = row['Dataset Title']
        
    dfFields = pd.DataFrame(fields_sheet.get_all_records(head=1))
    fields = {}
    for index,row in dfFields.iterrows():
        s4x4 = row["Socrata ID"]
        of = row["Source Field Name"]
        tf = row["Full Field Name"]
        af = row["API Field Name"]
        if s4x4 in fields:
            fields[s4x4]["source"].append(of)
            fields[s4x4]["cim"].append(tf)
            fields[s4x4]["api"].append(af)
        else:
            fields[s4x4] = {}
            fields[s4x4]["source"] = []
            fields[s4x4]["cim"] = []
            fields[s4x4]["api"] = []
            
            fields[s4x4]["source"].append(of)
            fields[s4x4]["cim"].append(tf)
            fields[s4x4]["api"].append(af)
        
        
        
    return xrefsBy4x4,xrefsByTitle,fields

#############################################

def go_deeper(aDict,value,nit):
    nit+=1
    for k, v in aDict.items():
         if not bool(v):            
             aDict[k] = []
             aDict[k].append(value)
         elif isinstance(v,list):
             aDict[k].append(value)
         else:
             go_deeper(v,value,nit)
   
    return aDict

###################################################    

def init():
    global dataSetsEtl,datasets,groupMenu
    groups = []
    desktop = pathlib.Path("/home/joe/bic_etl")
    runEtls = []
    dataSets = []
    info = {}
    # .rglob() produces a generator too
    desktop.rglob("*")
    files = list(desktop.rglob("*"))
# Which you can wrap in a list() constructor to materialize
    for ff in files:

            if (str(ff).split("/")[-1] == "run_etl.json"):     
                a=str(ff).split("/")
                m = a.index("bic_etl")
                b = a[m+1:-1]
                ll = splitL(b)
                groups.append(ll)
                runEtls.append(ff)
            
    print(f"{len(runEtls)} run_etl.json files found")    
    
    dataSets = []
    groups = []
    for file in runEtls:
      f = open(file,"r")
      data = json.load(f)
      # print(file)
      # print("----")
      a=str(file).split("/")
      m = a.index("bic_etl")
      b = a[m+1:-1]
      group = b[0]
      ll = splitL(b)
      bdir = "/".join(b)  
      print("GROUP ",group)
      
    #  groups.append(ll)
      for val in data:
            if "title" in val:
                  title=val["title"]
                  info[title]={}
                  info[title]["directory"] = bdir
                  info[title]["group"] = group
                
    #              print(title,ll)
                  nit=0
                  ll = go_deeper(ll,title,nit)
             #     info[title]["groups"] = ll
            
     #             print(ll)
     #             groups.append(ll)
            dataSets.append(val)
    #  print("FF ",ll) 
      groups.append(ll)

    datasets = []
    dataSetsEtl={}
    for val in dataSets:
        if "title" in val:
          datasets.append(val["title"])
          title=val["title"]
          dataSetsEtl[title] = {}  
          dataSetsEtl[title]["info"] = {}
          dataSetsEtl[title]["info"]["directory"] = info[title]["directory"]
          dataSetsEtl[title]["info"]["group"] = info[title]["group"]
     #     dataSetsEtl[title]["info"]["groups"] = info[title]["groups"]
            
        
            
          for k,v in val.items():
          #      print(k,v)
                if k != "title":
                    dataSetsEtl[title][k] = {}
                    if isinstance(v,dict):
                        for k1,v1 in v.items():
                            dataSetsEtl[title][k][k1]=v1
                            
                            
                            
    groupMenu = ['Groups',
     ['boulder',
      ['Restaurant Inspections in Boulder Colorado'],
      'catalog',
      ['CIM Catalog Download'],
      'cdhe',
      ['Enrollment Demographics for Post-Secondary Graduates in Colorado',
       'Post-Secondary Financial Aid Demographics in Colorado'],
      'cdor',[
      'revenue_marijuana',
      ['Marijuana Sales by County in Colorado',
       'State Retail Marijuana Sales Tax Revenue by County in Colorado',
       'Marijuana Tax and Fee Revenue in Colorado',
       'Marijuana Sales Revenue in Colorado'],
      'retail_reports',
      ['Retail Sales Tax Return History in Colorado',
       'Retail Reports by City in Colorado',
       'Retail Reports by County in Colorado',
       'Retail Reports by Industry and City in Colorado',
       'Retail Reports by Industry and County in Colorado',
       'Retail Reports by Industry in Colorado'],
      'regulations_liquor',
      ['Liquor Permits for Special Events in Colorado',
       'Liquor Compliance Check Statistics in Colorado',
       'Liquor Licenses in Colorado',
       'Recently Approved Liquor Licenses in Colorado',
       'Recently Expired and Surrendered Liquor Licenses in Colorado',
       'Sales Rooms in Colorado',
       'Manufacturer Temporary Sales Room Permits in Colorado']
      ],
      'cdos',[
      'health',
      ['Durable Medical Equipment Suppliers in Colorado'],
      'government',
      ['Current Notaries in Colorado'],
      'business',[
      'ucc',
      ['Uniform Commercial Code (UCC) Collateral Information in Colorado',
       'Uniform Commercial Code (UCC) Debtor Information in Colorado',
       'Uniform Commercial Code (UCC) Filing Information in Colorado',
       'Secured Party Information in Colorado'],
      'business',
      ['Business Entities in Colorado',
       'Business Entity Transaction History',
       'Trademarks for Businesses in Colorado',
       'Trade Names for Businesses in Colorado',
       'Master List in Colorado'],
      'nonprofit',
      ['Federal Tax-Exempt Subsection Codes in Colorado',
       'Registration for Charities, Paid Solicitors, Professional Fundraising Consultants, and for-profit Public Benefit Corporations in Colorado',
       'Charitable Organizations’ Offices in Colorado',
       'Other State Solicitation of Charities’ Registrants in Colorado',
       'Charitable Purpose of the Charity in Colorado',
       'Paid Solicitor Solicitation Notices in Colorado',
       'Campaign Reports for Solicitation Notices to Charities in Colorado',
       'Solicitation Campaign Supervisors Listed on Solicitation Notices in Colorado',
       'Charity Extension Requests',
       'Persons Associated with Charitable Organizations, Paid Solicitors, and Professional Fundraising Consultants in Colorado',
       'Other Names a Registered Entity Uses to Solicit Contributions',
       'Paid Solicitors Disclosed on Charity Registration Forms in Colorado',
       'Charitable Solicitation Call Center Locations in Colorado',
       'Charities Solicitation Type by Solicitation in Colorado',
       'Communication Methods Used in Solicitation Campaigns in Colorado']
      ],
      'lobbyist',
      ['Directory of Lobbyists in Colorado',
       'Directory of Lobbyist Clients in Colorado',
       'Expenses for Lobbyists in Colorado',
       'Characterization of Lobbyist Clients in Colorado',
       'Subcontractors for Lobbyists in Colorado',
       'Bill Information and Position with Income of Lobbyist in Colorado']
      ],
      'cdot',[
      'transportation_road_attributes',
      ['Highway Milepoints in Colorado',
       'Highway Mileposts in Colorado',
       'Highway Routes in Colorado',
       'Highway Routes in Colorado',
       'Local Roads in Colorado',
       'Major Roads in Colorado',
       'Scenic Byways in Colorado'],
      'tops',
      ['CDOT Expenses', 'CDOT Revenues', 'CDOT Payroll'],
      'natural_resources',
      ['Lakes in Colorado', 'Streams in Colorado'],
      'transportation_infrastructure',
      ['Airports in Colorado',
       'Cities in Colorado',
       'Counties in Colorado',
       'Railroads in Colorado']
      ],
      'ceo',[
      'useia',
      ['Gasoline Prices in Colorado', 'Natural Gas Prices in Colorado']
      ],
      'denver',
      ['Temporary Outdoor Expansions for Restaurants in Denver, Colorado'],
      'dola',[
      'special_districts',
      ['Metro Districts in Colorado',
       'Parks and Rec Districts in Colorado',
       'Fire Districts in Colorado',
       'Hospital Districts in Colorado',
       'Water and Sanitation Districts in Colorado',
       'Library Districts in Colorado',
       'School Districts in Colorado',
       'Soil Districts in Colorado',
       'Cemetery Districts in Colorado',
       'All Special Districts in Colorado'],
      'boundaries',
      ['Municipal Annexations in Colorado', 'Municipal Boundaries in Colorado'],
      'demographics',
      ['Population Projections in Colorado',
       'Race Estimates in Colorado',
       'Race Forecast in Colorado']
      ],
      'dora',[
      'regulations',
      ['Licensed Real Estate Professionals in Colorado',
       'Professional and Occupational Licenses in Colorado']
      ],
      'dpa',
      ['DPA Tops Data'],
      'irs',
      ['Purpose and Operational Size of Charities Operating in Colorado',
       'Fundraising Revenue of Charities Operating in Colorado',
       'Total Revenue of Charities Operating in Colorado',
       'IRS Filing Information for Charities Operating in Colorado',
       'Total Revenue and Types of Art for Charities Operating in Colorado',
       'Conservation Easements for Charities Operating in Colorado',
       'Activities of Charities Operating in Colorado',
       'Expenses of Charities Operating in Colorado',
       'Expenses of Charities Operating in Colorado'],
      'tchd',
      ['Restaurant Inspections in Tri-County Colorado']]]

    return sorted(datasets)

######################################################################

def map4x4(row):
    
    if isinstance(row["Data Link"],str) and  len(row["Data Link"]) == 9 and re.findall( "\w{4}-\w{4}",row["Data Link"]):
  #      link = '<a href="https://data.colorado.gov/dataset/{}">{}</a>'.format(row["Data Link"],row["Data Link"])
        
        link = 'https://data.colorado.gov/dataset/{}'.format(row["Data Link"])
    else:
        link = ""
        
    return link
    
#####################################################################

def runETL(k):
    global extract,a,string,dataSetsEtl
    text2 = ""
  #  print(k)
    directory = ""
    group = ""
    if "info" in dataSetsEtl[k]:
        if "directory" in dataSetsEtl[k]["info"]:
            directory = dataSetsEtl[k]["info"]["directory"]
            group = dataSetsEtl[k]["info"]["group"]
            
## Build Summary Text string
    for ky1 in dataSetsEtl[k].keys():
        text2+=f"{ky1}:\n" 
        for k2,v2 in dataSetsEtl[k][ky1].items(): 
            sl = 10 - len(k2)
            s = " "*sl
        
            if isinstance(v2,list) == False:
                text2+=f"    {k2:10s}{s} :  {v2}\n"
            else:
                text2+=f"    {k2:10s}{s} : {v2[0]}\n"
                if len(v2) > 1:
                    for val in v2[1:]:
                        text2+= f"                   {s} : {val}\n"
        text2+=f"\n\n"
## Build actions
    extract = ""
    if 'extract' in dataSetsEtl[ds]:
        if (dataSetsEtl[ds]['extract']['language'] == 'node'):
            pgm =  dataSetsEtl[ds]['extract']['file']
            options=""
            if 'options' in dataSetsEtl[ds]['extract']:

                for opts in dataSetsEtl[ds]['extract']['options']:
                    options+= f" {opts}" 
            extract = f"node {bicHome}{pgm} {options}"
        #    print(extract)
    transform=""    
    if 'transform' in dataSetsEtl[ds]:
        
        if dataSetsEtl[ds]["transform"]["language"] == "node":
            ff=dataSetsEtl[ds]["transform"]["file"]
            transform=f"node {bicHome}{directory}/{ff}"
            
        else:
            file=""        
        
    text2+=f"\n\nActions Strings\nExtract\n{extract}\n\nTransform\n{transform}"
        
    return text2,extract,transform
    
#############################################

def setRowColors(lst,col1,col2,colsp,header):
    count=0
    colors = {}
    # print("SRC head ",header)
    # print("SRC list",lst)

    nrec = header.index("% Missing")
    for vals in lst:
        key = vals[0]
        if count%2 == 0:
            colors[key] = col1
        else:
            colors[key] = col2
        if vals[nrec] > 99.0:
           
            colors[key] = colsp
        count+=1
    colTab = []
    for key,colr in colors.items():
        colTab.append(colr)
    rowNums = [num for num in range(0,len(colTab)+1)]
    colText = ["black"]*len(colTab)
    colrw = list(zip(rowNums,colTab))
  
    return colrw

##########################################################

def setRowColorsGeneric(lst,col1,col2):
    count=0
    rowNums=[]
    colTab=[]

    for vals in lst:       
        if count%2 == 0:
            colTab.append(col1)
        else:
            colTab.append(col2)
        rowNums.append(count)
        count+=1
    colrw = list(zip(rowNums,colTab))
  
    return colrw

###########################################################

def showDfRecs(df1,colUn,val,windowP,title="DF Unique Record Values"):
    global color1,color2
    header_list = list(df1.columns)
  
    col_widths = [8]*len(header_list)
 #   col_widths[0] = 25
    columnSortStateTable1 = {}
  

    ## set up sort state for the column in both tables
    for col in header_list:
        columnSortStateTable1[col] = -1
       
            
#    valsFile1 = getValues(stats1,header_list)
    values = df1.values.tolist()
    
    rowFile1Colors = setRowColorsGeneric(values,color1,color2)
    
    layout = [[sg.Text(f"Column: ",font="CENTAUR 10"),
               sg.Text(f"{colUn}",font="CENTAUR 15")],
              [sg.Text(f"Unique Value: ",font="CENTAUR 10"),
               sg.Text(f"{val}",font="CENTAUR 15")],
               [sg.Button("Quit")],
               [sg.Table(values=values,text_color="black", auto_size_columns=False,enable_events=True,num_rows=20,col_widths=col_widths,font="CENTAUR 10",
                   justification='center',pad=(5,5),vertical_scroll_only=False,
                   key='-TABLEFILE-',row_colors=rowFile1Colors,headings = header_list,metadata=columnSortStateTable1)]
               ]

    colorTheme = windowP.metadata[0]          
#    sg.theme(colorTheme)     
    window2 = sg.Window(title,layout,finalize=True,resizable=True,metadata=[colorTheme,df1])
    a = window2.CurrentLocation()
    screen_width, screen_height = window2.get_screen_dimensions()
    win_width, win_height = window2.size
    x, y = (screen_width - win_width)//2, (screen_height - win_height)//2
    x=200
    y=200
    window2.move(x, y)
    tableFile1 = window2['-TABLEFILE-']
    tableFile1.bind('<Button-1>', "Click")
    headerStore[tableFile1] = header_list
    return window2,tableFile1,values,rowFile1Colors

###################################################

def splitL(data):
    if data:
        head, *tail = data  # This is a nicer way of doing head, tail = data[0], data[1:]
        return {head: splitL(tail)}
    else:
        return []

#############################################

def stringArgs(string):
#  This is set to decode the extract string to get the location and name of the 
#  extracted data file
    h =split(string)
    inf = h.index("-f")
    inf = h[inf+1]
    ot = h.index("-a")
    ot = h[ot+1]
    of = h.index("-o")
    of = h[of+1]
    inf=inf.replace(inf[-4:],ot)
    f = inf.split("/")
    finalFile = f"{bicHome}{of}{f[-1]}"
    return finalFile

######################################################    

def sortTable(row,stats,table,event,header):
    
        e = table.user_bind_event 
        region = table.Widget.identify('region', e.x, e.y)
        if region == 'heading':
            row = 0
        elif region == 'cell':
            row = int(table.Widget.identify_row(e.y))
   
        if row == 0:
            colSortState = table.metadata
            colClicked = int(table.Widget.identify_column(e.x)[1:])
            header = headerStore[table]
            
            statClicked = header[colClicked-1].strip()
           
            colSortState[statClicked]*=-1
            if colSortState[statClicked] == -1:
                sortAsc=False
            else:
                sortAsc=True
            if colClicked > 1:  # user number sort
                statsS = dict(sorted(stats.items(), key=lambda x: x[1][statClicked],reverse=sortAsc))
            else:
                statsS = dict(sorted(stats.items(), key=lambda x: x[0],reverse=sortAsc))


            statsVals=[]
            for col in statsS:
                 vals=[]
                 vals.append(col)
                 for k in header[1:]:
                    vals.append(statsS[col][k.strip()])
                 statsVals.append(vals)
           # slen= len(statsVals)
#             colorsTable = setRowColors(statsVals,"#b3f0ff","#33d6ff","pink",header_list)

#             window['-TABLE-'].update(values=statsVals,row_colors=colorsTable)
        
        return statsVals,colSortState


####################################################################

def showDFUN(col,unq,windowParent,colr1,colr2,colorTheme,title=""):
    global dataStore
    valuesUNQ = list(zip(unq.index.tolist(),unq.tolist()))
    hUNQ = []
    hUNQ.append("Values")
    hUNQ.append("Count")

    
    sortState = {}
    for val in hUNQ:
        sortState[val]=-1
    layout2 = [    
                   [sg.Text(f"Showing unique Values for Column"),sg.Text(f" {col}",text_color="white",font='Courier 15 bold '),sg.Text(f" and Reg Ex",text_color="white",font='Courier 10 bold ')],
                
                   [sg.Button('Quit')],
            
                   [sg.Button('Write Unique'),
                    sg.Table(values=valuesUNQ,
                       background_color=colr1,vertical_scroll_only=False,col_widths=60,font='Courier 10 bold ' ,
                       auto_size_columns=True,enable_events=True,def_col_width=25,text_color="black",
                       justification='right',alternating_row_color=colr2,
                       key='-TABLE-', headings = hUNQ,metadata=sortState)]
            ]
    sg.theme(colorTheme)    
    window2 = sg.Window(f"Unique for {title}", layout2,finalize=True,resizable=True, grab_anywhere=False,metadata=[windowParent,col])

    table = window2['-TABLE-']
    table.bind('<Button-1>', "Click")
    dataStore[table]=valuesUNQ
    
    return window2,table,valuesUNQ,hUNQ

############################################################

def showColAnal(xrefsO2T,xrefsT2O,colr1,colr2):
    global dataStore
    values=[]
    header = ["Original Column","Transformed Column"]
    for col in sorted(xrefsO2T.keys()):
        tmp = [col,xrefsO2T[col]]
        values.append(tmp)
    
    rowColors = setRowColorsGeneric(values,colr1,colr2)
    
    sortState = {}
    for val in header:
        sortState[val]=-1
        
    
    a = [[sg.Text("Columns in Program NOT in Data",font='Courier 10 bold ',justification="left")],
         [sg.Listbox("",font='Courier 15 bold ',horizontal_scroll=True ,size=(20,5),key="-PNOTD-")]] 
    
    d = sg.Column(a)
    
    b = [[sg.Text("Columns in Data NOT in Program",font='Courier 10 bold ',justification="left")],
         [sg.Listbox("",font='Courier 15 bold ' ,enable_events=True,horizontal_scroll=True ,size=(20,5),key="-DNOTP-")]] 
    e = sg.Column(b)
    
    c = [d,sg.VerticalSeparator(color="black"),e]
    layout2 = [    
                   [sg.Text(f"Original COlumns Transformed to Columns")],
                   [sg.Button("Compare 2 Data"),sg.Button("Get SIJ Cols"),sg.Button("Full Data Column Map")],
                   [sg.Button('Quit')],
                   [c],
                   [ sg.Table(values=values,
                       vertical_scroll_only=False,col_widths=60,font='Courier 10 bold ' ,
                       auto_size_columns=True,enable_events=True,def_col_width=25,text_color="black",
                       justification='right',row_colors=rowColors,
                       key='-TABLECOLUMN-', headings = header,num_rows=20,
                       metadata=[values,rowColors,sortState])]
                ]
#    sg.theme(colorTheme)    
    window2 = sg.Window(f"Orig vs Trans Column Analysis", layout2,finalize=True,resizable=True, grab_anywhere=False,metadata="nothing")

    table = window2['-TABLECOLUMN-']
    table.bind('<Button-1>', "Click")
    dataStore[table]=values
    headerStore[table]=header
    windowsStore["columnAnalysis"] = window2   
    return window2,table,values,header

##########################################################

def showFullDataColMap(cols,vals):
    
    layout2 = [    
                   [sg.Text(f"Full Data Columns Map")],
                   [sg.Button('Quit')],
                   [sg.Button('Show Missing Fields')],
                   [ sg.Table(values=vals,
                       vertical_scroll_only=False,col_widths=60,font='Courier 15 bold ' ,
                       auto_size_columns=True,enable_events=True,def_col_width=25,text_color="black",
                       justification='right',row_colors=rowColors,
                       key='-TABLEDCMAP-', headings = cols,num_rows=20,
                       metadata=[vals])]
                ]
        
    window2 = sg.Window(f"Orig vs Trans Column Analysis", layout2,finalize=True,resizable=True, grab_anywhere=False,metadata="nothing")

    table = window2['-TABLEDCMAP-']
    table.bind('<Button-1>', "Click")
    dataStore[table]=values
    headerStore[table]=cols
    windowsStore["fullDataColMap"] = window2  

###########################################################

def showMissingColumns():
    global missed

    header_list = ["Column","% Missing","Missing","string","integer","float","boolean"]
    stats1Ex = {}
    
    stats1Tr = {}
    stats1Ci = {}
    
    for col in missed['Extract - Data']:
        print(col,stats1Extract[col])
        stats1Ex[col]= stats1Extract[col]

    
    for col in missed['Transform - Data']:
        print(col,stats1Transform[col])
        stats1Tr[col]= stats1Transform[col]

    for col in missed['CIM - Data']:
        print(col,stats1Cim[col])
        stats1Ci[col]= stats1Cim[col]
  
    valsExtract = getValues(stats1Ex,header_list)
    valsTransform = getValues(stats1Tr,header_list)
    valsCim = getValues(stats1Ci,header_list)
    
    
    
    layout = [
               [sg.Button("Quit")],
               [sg.Listbox(values=missed["BIC Inventory"],size=(10,5),font="CENTAUR 10"),
                sg.Listbox(values=missed["Transform Program"],size=(10,5),font="CENTAUR 10"),
                sg.Listbox(values=missed["Transform - SIJ"],size=(10,5),font="CENTAUR 10")],
                [sg.Table(values=valsExtract,text_color="black", auto_size_columns=False,enable_events=True,num_rows=10,font="CENTAUR 10",
                    justification='center',pad=(5,5),vertical_scroll_only=False,
                    key='-TABLEMISSEXTR-',headings = header_list)],
                [sg.Table(values=valsTransform,text_color="black", auto_size_columns=False,enable_events=True,num_rows=10,font="CENTAUR 10",
                    justification='center',pad=(5,5),vertical_scroll_only=False,
                    key='-TABLEMISSTRANS-',headings = header_list)],
                [sg.Table(values=valsCim,text_color="black", auto_size_columns=False,enable_events=True,num_rows=10,font="CENTAUR 10",
                    justification='center',pad=(5,5),vertical_scroll_only=False,
                    key='-TABLEMISSCIM-',headings = header_list)]
               ]

#    colorTheme = windowP.metadata[0]  
    theme = "LightBrown 3"
    sg.theme("LightBrown 3")     
    window2 = sg.Window("Missing Fields",layout,finalize=True,resizable=True,metadata=[theme])
    a = window2.CurrentLocation()
    screen_width, screen_height = window2.get_screen_dimensions()
    win_width, win_height = window2.size
    x, y = (screen_width - win_width)//2, (screen_height - win_height)//2
    x=200
    y=200
    window2.move(x, y)
    tableex = window2['-TABLEMISSEXTR-']
    tableex.bind('<Button-1>', "Click")
    tabletr = window2['-TABLEMISSTRANS-']
    tabletr.bind('<Button-1>', "Click")
    tableci = window2['-TABLEMISSCIM-']
    tableci.bind('<Button-1>', "Click")
    
    dataStore[tableex] = valsExtract
    dataStore[tabletr] = valsTransform
    dataStore[tableci] = valsCim
    
    
    return window2,tableex,tabletr,tableci
    # headerStore[tableFile1] = header_list
    
 
###########################################################

def sortUniqe(table,window,dataStore,headerStore):
    global color1,color2
    try:
        e = table.user_bind_event
        region = table.Widget.identify('region', e.x, e.y)
        sortAsc = {}
        sortAsc[1] =False
        sortAsc[-1]=True
   #     print("R ",region)
        if region == 'heading':
            values = dataStore[table]
            
           
            header = headerStore[table]
            # print("SU header ",header)
            column = int(table.Widget.identify_column(e.x)[1:])
            col=header[column-1]
            
            sortState = table.metadata
           
            sortState[col]*=-1
            table.metadata = sortState
          
            values = sorted(values, key=lambda element: (element[column-1]),reverse=sortAsc[sortState[col]]) 
#            rowFile1Colors = setRowColors(values,color1,color2,"pink",header)
            rowFile1Colors = setRowColorsGeneric(values,color1,color2)

            window["-TABLE-"].update(values=values)
            dataStore[table] = values
    except Exception as err:
        exceptionLog(err,inspect.currentframe().f_code.co_name)
                    
#####################################################        

def tranfColXref(file):
    global transformProgram
 #   fin = open("/home/joe/bic_etl/cdos/business/nonprofit/scripts/reg_finan.js","r")
    fin = open(file,"r")
    
    lines = fin.readlines() 
    transformProgram = lines
    fout = open("output.txt","w")
    org = []
    xrefsO2T = {}
    xrefsT2O = {}

    for line in lines:
        if re.findall("row",line.lower()) and re.findall("=",line.lower()) and not re.findall("^//",line.lstrip()):
            st1=line.find("[")
            ed1=line.find("]")
            st2=line.rfind("[")
            ed2=line.rfind("]")
            org.append(line[st2+2:ed2-1])
    #        print(line)
    #        print(line[st1+1:ed1],line[st2+1:ed2])
            fout.write(f"{line[st1+1:ed1]}  {line[st2+1:ed2]}\n")
            og = line[st2+1:ed2].replace("'","")
            og = og.replace('"','')
            
            og = og.replace("].toLowerCase()","")


            tr = line[st1+1:ed1].replace(".toLowerCase()","")
            tr = tr.replace('"','')
            tr = tr.replace("'","")
            



            # tr = line[st1+1:ed1]
            # og = line[st2+1:ed2]

            xrefsT2O[tr]  = og
            xrefsO2T[og]  = tr
    return xrefsO2T,xrefsT2O
                
############################################################

def wrap(string, lenght=60):
    if isinstance(string,str):
       return '\n'.join(textwrap.wrap(string, lenght))
    else:
        return ""
        
############################################################    
       
def cronGUI():
    global ds,text2,a,string,numWindows,color1,color2,file,transform,dataStore,statsStore
    global windowsStore,xrefsBy4x4,groupMenu
    global transformProgram,dnotpLines
    global extractColumns,transformColumns,cimColumns
    global transformSijColumns,loadSijColumns,xrefsO2T,xrefsT2O
    global dfExtract,dfTransform,dfCim,missed,fields
    global stats1Extract,stats1Transform,stats1Cim
    
    dataStore = {}
    statsStore = {}
    windowsStore = {}
    datasets = init()
    cimColumns = []
    transformColumns = []
    extractColumns = []
    transformSijColumns = []
    loadSijColumns = []
    
    
##  get xrefs b/w 4x4 ids and datasert titles...yay   
    xrefsBy4x4,xrefsByTitle,fields = getXrefs()

    mf = pd.read_excel("BICDataInventoryandMetadata.xlsx",skiprows=0,sheet_name="Inventory_Active",engine="openpyxl")
    
    mf["CIM Link"] = mf.apply(map4x4,axis=1)

    mf=mf.loc[~mf["Standardized Title for Dataset"].isna()]
    mfShort = mf[['Standardized Title for Dataset', 'Data Link','CIM Data Type','GoCodePublishYear','Standardized Short Description','CIM Link']]
    mfShort["Standardized Title for Dataset"] = mfShort["Standardized Title for Dataset"].astype(str)   
    
    header_list = list(mfShort.columns)
    # mfV = []
    # a = ["","",nan,nan,""]
    # mfV.append(a)
    layout = [
              [sg.Text("BIC Cron DataSets")],
              [sg.Combo(datasets,enable_events=True,key="-DATASET-",font='Courier 10 bold ')],
              [sg.ButtonMenu('Groups', menu_def=groupMenu, key='Group Menu')],
              [sg.Text("4x4",font='Courier 15 bold '),sg.Input("wwbh-7bpa",size=[8,1],key="-4x4-",font='Courier 10 bold '),
               sg.Button("4x4")],
              [sg.Button('Close'),sg.Button("Plot Crons")],
              [sg.Button("Column Analysis")],
   #          [sg.Button("GO")],
              [sg.Button("Extract",font='Courier 10 bold ',key="-EXTRACT-",visible=True),
               sg.Button("Analyze-Extr",font='Courier 10 bold ',key="-EXTRACTANAL-",visible=False),
               sg.Text("",visible=False,font='Courier 15 bold ',key="-EXTRACTINFO-")],
              [sg.Button("Transform",visible=False,font='Courier 10 bold ',key="-TRANSFORM-"),
               sg.Button("Ananlyze-Trans",font='Courier 10 bold ',key="-TRANSFORMANAL-",visible=False),
               sg.Text("",visible=False,font='Courier 15 bold ',key="-TRANSFORMINFO-")
              ],
              [sg.Button("CIM API",font='Courier 15 bold ',key="-CIMAPI-",visible=False),sg.Text("",visible=False,font='Courier 15 bold ',key="-CIMINFO-")],
              [sg.Text(text="Dataset Summary",enable_events=True,font='Courier 15 bold ',background_color="blue",key="-OUTPUT-",size=[70,10]),
               sg.Multiline(default_text="ETL Summary",key="-ETL-",size=[70,20],font='Courier 15 bold ')]
            ]

       
    # Create the Window
    sg.theme('Dark Green 5')
    window2 = sg.Window('ETL', layout,finalize=True,resizable=True)
    #window = sg.Window('Window Title', layout, web_port=2222, web_start_browser=False)
    #table = window2['-TABLE2-']

    #window2.move(window.current_location()[0]+600, window.current_location()[1])
    try: 
        while True:

         #   event, values = window2.read()
            wid, event, values = sg.read_all_windows()
            print('event: ',event, 'values: ',values)
 
            if event == sg.WIN_CLOSED or event == 'Close':
                window2.close()
                break
            elif event ==  event == 'Quit':
                wid.close()
### DATASET                
            elif event == "-DATASET-" or event == "4x4" or event == "Group Menu":
      #      elif event == "GO":#
                if event == "-DATASET-":
                    (ds)  = list(values.items())
                    ds=ds[0][1]
                elif event == "4x4":
                    ds = xrefsBy4x4[values["-4x4-"]]
                    print("XREF DS ",values["-4x4-"],ds)
                elif event == "Group Menu":
                    ds = values["Group Menu"]
  #              ds = "Paid Solicitors Disclosed on Charity Registration Forms in Colorado"
                print("Daetaset ",ds)
                mmf = mfShort.loc[mfShort["Standardized Title for Dataset"].str.strip() == ds.strip()]
                cim4x4 = mmf['Data Link'].values.tolist()[0]
                cimApi = f"https://data.colorado.gov/api/views/{cim4x4}/rows.csv?accessType=DOWNLOAD"
                if mmf.shape[0] > 0:
                #            mmf["Standardized Short Description"] = mmf["Standardized Short Description"].map(wrap)          
                    text = f"Title             : {mmf['Standardized Title for Dataset'].values.tolist()[0]}\nSocrata        : {mmf['Data Link'].values.tolist()[0]}\nData Type    : {mmf['CIM Data Type'].values.tolist()[0]}\nPublish Year: {mmf['GoCodePublishYear'].values.tolist()[0]}\nCIM Link : {mmf['CIM Link'].values.tolist()[0]}\nDescription  : {mmf['Standardized Short Description'].values.tolist()[0]}"            
                   # display(mmf)
                else:
                    text = "None"
                if ds in dataSetsEtl:
                      text2,extract,transform = runETL(ds)
                else:
                      text2=""
                      extract = ""
                if len(extract) > 10:
                    window2["-EXTRACT-"].update(visible=True)
                if len(transform) > 10:
                    window2["-TRANSFORM-"].update(visible=True)
                    window2["-TRANSFORM-"].update(visible=True)
          
                if len(cimApi) > 10:                 
                    text2+=  f"\n\nCIM API\n{cimApi}"
                    window2["-CIMAPI-"].update(visible=True)
                    
                   
                window2["-OUTPUT-"].update(text)
                window2["-ETL-"].update(text2)
                link =  mmf['CIM Link'].values.tolist()[0]
           #     print("Link ",mmf['CIM Link'],link.find("http"))
                
                # if link.find("http") > -1:
                #      window2["-LINK-"].update (visible=True)
                # else:
                #      window2["-LINK-"].update (visible=False)
### LINK                
               
            elif event == "-LINK-":
                    print("Going To: ",link) 
                    if len(link) > 10:
                        webbrowser.open(link)
### Plot Crons                
            elif event == "Plot Crons":    
                plotCrons(crons_all)

### Show Missing Columns
            elif event == "Show Missing Fields":
                  window2,tableex,tabletr,tableci = showMissingColumns()
### Full Data Column Map
            elif event == "Full Data Column Map":
                cols,vals,missed = fullDataColumnMap(cim4x4)
                showFullDataColMap(cols,vals)
            
### Get SIJ Cols
            elif event == "Get SIJ Cols":
                getSijColumns("")
                print("Tr SIJ ",transformSijColumns)
                print("LD SIJ ",loadSijColumns)
                 
### DNOTP

            elif event == "-DNOTP-":
                col = values["-DNOTP-"][0]
                if col in dnotpLines:
                    string=""
                    string = "\n".join(dnotpLines[col])
                else:
                    string="NOTHING FOUND"
                sg.Popup(string)
                    
                print("DNOTP STRai",string)
### Column Analysis                
            elif event == "Column Analysis":
           #     trfile = dataSetsEtl[ds]["transform"]["file"]
                trfile = transform[5:]
                
                xrefsO2T,xrefsT2O = tranfColXref(trfile)
                color1 = colorPairs[numWindows][0]
                color2 = colorPairs[numWindows][1]
                
                numWindows+=1
                if numWindows > len(colorPairs):
                    numWindows=0
                showColAnal(xrefsO2T,xrefsT2O,color1,color2)
             
### Compare 2 Data                 
            elif event == "Compare 2 Data":   
                print("HERE We GO")
                ww = windowsStore["extract"]
                wdf = ww.metadata[1]
                
                compare2Data(wdf,xrefsO2T)
                         
### CIMAPI                
            elif event == "-CIMAPI-":
                  #  df1,stats1 = getFilesClicked(cimApi,window2)
                    dfCim,stats1Cim = getFile("Fetch",cimApi,window2)
                    cimColumns = list(dfCim.columns)
                    color1 = colorPairs[numWindows][0]
                    color2 = colorPairs[numWindows][1]

                    numWindows+=1
                    if numWindows > len(colorPairs):
                        numWindows=0

                    WindowC,tabl1,valsFile1,rowFile1Colors = compareWindow(stats1Cim,dfCim,cimApi,"CMI Analysis","DarkRed")
                    dataStore[tabl1]=valsFile1
                    
                    windowsOpen["main"].append(WindowC)  
                    
### TRANSFORM               
            elif event == "-TRANSFORM-":
               
                a=subprocess.run(transform,shell=True,capture_output=True,text=True)
                string= "\n\n-------------------------------------\n"
                string+= "TRANSFORM Output\n"
                tfile=""
                if len(a.stderr) > 0:
                    string+= f"Error:\n {a.stderr}"
                else:
                    for msg in a.stdout.split("debug msg:"):    
                        if msg.find("final file is at:") > -1:
                           
                            tmp = msg.split(" ")
                            tfile = tmp[-3]
                            if os.path.isfile(tfile):
                                
                                window2["-TRANSFORMANAL-"].update(visible=True)
                                fileStats = os.stat(tfile)
                                dt = datetime.fromtimestamp(fileStats.st_ctime)
                                string = f"{fileStats.st_size:,} bytes Created: {dt}  file: {tfile}"
                                window2["-TRANSFORMINFO-"].update(string,visible=True)
                            
                    string+=f"STDOUT: {a.stdout}"
                    window2["-ETL-"].update(string,append=True)
                    
### TRANSFORMANAL
            elif event == "-TRANSFORMANAL-":
                #    df1,stats1 = getFilesClicked(tfile,window2)
                    dfTransform,stats1Transform = getFile("Local",tfile,window2)
                    transformColumns = list(dfTransform.columns)
                    color1 = colorPairs[numWindows][0]
                    color2 = colorPairs[numWindows][1]
                  
                    numWindows+=1
                    if numWindows > len(colorPairs):
                        numWindows=0

                    WindowC,tabl1,valsFile1,rowFile1Colors = compareWindow(stats1Transform,dfTransform,tfile,"Transform Analysis","DarkPurple")
                    dataStore[tabl1]=valsFile1
                    windowsOpen["main"].append(WindowC)    
                    
### EXTRACT                  
            elif event == "-EXTRACT-":
                a=subprocess.run(extract,shell=True,capture_output=True,text=True)
                string= "\n\n-------------------------------------\n"
                string+= "Extract Output\n"
                if len(a.stderr) > 0:
                    string+= "fError:\n {a.stderr}"
                string+=f"STDOUT: {a.stdout}"
                window2["-ETL-"].update(string,append=True)
                sx = string.find("successfully download to")
                string[sx+25:]
                sxo = string[sx+25:].find("!")
             
                file=f"/home/joe/bic_etl{string[sx+25:sx+25+sxo]}"
              
                if os.path.isfile(file):
                  
                    window2["-EXTRACTANAL-"].update(visible=True)
                    fileStats = os.stat(file)
                    dt = datetime.fromtimestamp(fileStats.st_ctime)
                    string = f"{fileStats.st_size:,} bytes Created: {dt}  file: {file}"
                    window2["-EXTRACTINFO-"].update(string,visible=True)
                    
### EXTRACTANAL                      
            elif event == "-EXTRACTANAL-":
                file=stringArgs(extract)
                
             #   df1,stats1 = getFilesClicked(file,window2)
                dfExtract,stats1Extract = getFile("Local",file,window2)
                extractColumns = list(dfExtract.columns)
                color1 = colorPairs[numWindows][0]
                color2 = colorPairs[numWindows][1]
                
                numWindows+=1
                if numWindows > len(colorPairs):
                    numWindows=0
                    
                WindowC,tabl1,valsFile1,rowFile1Colors = compareWindow(stats1Extract,dfExtract,file,"Extract Analysis","DarkBlue16")
                dataStore[tabl1]=valsFile1
                windowsOpen["main"].append(WindowC)
                windowsStore["extract"] = WindowC
                
### TABLEFILE-CLICK
            elif event == "-TABLEFILE-Click":
               
                row,col=getRowClicked(tabl1,valsFile1)
               
                if row == 0:
                   header = headerStore[tabl1]
                   tabl1 = wid["-TABLEFILE-"]
                   stats1 = statsStore[tabl1]
                   valsFile1,columnSortStateTable1 = sortTable(row,stats1,tabl1,event,header)          
                   rowFile1Colors=setRowColors(valsFile1,color1,color2,"pink",header)
                   wid["-TABLEFILE-"].update(values=valsFile1,row_colors=rowFile1Colors)
                   wid["-TABLEFILE-"].metadata=columnSortStateTable1
                else:
                    df1 = wid.metadata[1]
                    unq = df1[col].value_counts()
                    colorTheme=wid.metadata[0]
                    w,t,values,uHead = showDFUN(col,unq,wid,color1,color2,colorTheme)
                  #dataStore[t] = values
                    headerStore[t] = uHead
                    windowsOpen["main"].append(w)
                    
### TABLE-Click
            elif event == "-TABLE-Click":  # This is the Unique Values tables
                table = wid['-TABLE-']               
               
                
                row,val =getRowClickedUN(table)
                if row == 0:
                    sortUniqe(table,wid,dataStore,headerStore)
                else:
                    widP = wid.metadata[0]
                    col = wid.metadata[1]
                    df = widP.metadata[1]
                    tmp = df.loc[df[col] == val]
                    showDfRecs(tmp,col,val,widP,title="DF Unique Record Values")
                    
###  -TABLEMISSEXTR-Click                   
            elif event in ["-TABLEMISSEXTR-Click","-TABLEMISSTRANS-Click","-TABLEMISSCIM-Click"] :
                if event == "-TABLEMISSEXTR-Click":
                   tabl1 = wid["-TABLEMISSEXTR-"]
                   title = "EXTRACT"
                   df = dfExtract
                elif event == "-TABLEMISSTRANS-Click":
                   tabl1 = wid["-TABLEMISSTRANS-"]
                   df =  dfTransform
                   title = "TRANSFORM"
                elif event == "-TABLEMISSCIM-Click":
                   tabl1 = wid["-TABLEMISSCIM-"]
                   df =  dfCim
                   title="CIM"
                 
                
                vals = dataStore[tabl1]
                row,col=getRowClicked(tabl1,vals)
                unq = df[col].value_counts()
                colorTheme=wid.metadata[0]
                w,t,values,uHead = showDFUN(col,unq,wid,color1,color2,colorTheme,title)
                  #dataStore[t] = values
                headerStore[t] = uHead
                windowsOpen["main"].append(w)
                   
         
    except  Exception as err: 
       exceptionLog(err,inspect.currentframe().f_code.co_name)
 
    for wid in windowsOpen["main"]:
        print("window MA",wid)
        if wid:
            print("cosing ",wid)
            wid.close()
            wid = None

cronGUI()



### Bottom

26 run_etl.json files found
GROUP  catalog
GROUP  cdos
GROUP  cdos
GROUP  cdos
GROUP  cdos
GROUP  cdos
GROUP  cdos
GROUP  cdhe
GROUP  dpa
GROUP  tchd
GROUP  boulder
GROUP  denver
GROUP  cdot
GROUP  cdot
GROUP  cdot
GROUP  cdot
GROUP  irs
GROUP  dola
GROUP  dola
GROUP  dola
GROUP  cdor
GROUP  cdor
GROUP  cdor
GROUP  ceo
GROUP  general
GROUP  dora
GGGGG  {'id': '1pVj8GwL13QtiyX7qG1xWf0kLacZwG7gV2wGT4xqmHCE', 'name': 'BIC Data Inventory and Metadata', 'createdTime': '2023-06-14T16:13:40.567Z', 'modifiedTime': '2023-08-17T03:30:49.859Z'}
GGGGG  {'id': '1WTaOglzbSsYiHhAGguGxHQXmAGmOhfFHkGkMLowxAOA', 'name': 'BIC Data Inventory and Metadata-OLD', 'createdTime': '2021-05-19T20:34:56.598Z', 'modifiedTime': '2023-06-14T17:17:41.383Z'}


/tmp/ipykernel_5968/735637673.py:1235: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mfShort["Standardized Title for Dataset"] = mfShort["Standardized Title for Dataset"].astype(str)


event:  4x4 values:  {'-DATASET-': '', 'Group Menu': None, '-4x4-': 'w6kb-3vsj', '-ETL-': 'ETL Summary'}
XREF DS  w6kb-3vsj Communication Methods Used in Solicitation Campaigns in Colorado
Daetaset  Communication Methods Used in Solicitation Campaigns in Colorado
event:  -EXTRACT- values:  {'-DATASET-': '', 'Group Menu': None, '-4x4-': 'w6kb-3vsj', '-ETL-': 'info:\n    directory   :  cdos/business/nonprofit\n    group           :  cdos\n\n\nextract:\n    language     :  node\n    file             :  general/scripts/sftp_extract.js\n    options       : -f charity/sol_typ_entity.txt\n                       : -o cdos/business/nonprofit/data_source/\n                       : -a .tsv\n\n\ntransform:\n    language     :  node\n    file             :  scripts/sol_typ_entity.js\n\n\nload:\n    file             :  sol_typ_entity.sij\n    type             :  datasync\n    format         :  csv\n    load_file   :  sol_typ_entity.csv\n\n\n\n\nActions Strings\nExtract\nnode /home/joe/bic_etl/genera

In [ ]:
sg.theme_previewer()

In [ ]:
https://docs.google.com/spreadsheets/d/1Xc7oYpdCLwHmUCaokpfXkF4bzkwRW0xUbvBZwruF0ZI/edit?usp=sharing

## Bottom

# stats1Extract

In [ ]:
missed

In [ ]:
def init():
    global dataSetsEtl,datasets,groups
    groups = []
    desktop = pathlib.Path("/home/joe/bic_etl")
    runEtls = []
    dataSets = []
    info = {}
    # .rglob() produces a generator too
    desktop.rglob("*")
    files = list(desktop.rglob("*"))
# Which you can wrap in a list() constructor to materialize
    for ff in files:

            if (str(ff).split("/")[-1] == "run_etl.json"):     
                a=str(ff).split("/")
                m = a.index("bic_etl")
                b = a[m+1:-1]
                ll = splitL(b)
                groups.append(ll)
                runEtls.append(ff)
            
    print(f"{len(runEtls)} run_etl.json files found")    
    
    dataSets = []
    groups = []
    for file in runEtls:
      f = open(file,"r")
      data = json.load(f)
      # print(file)
      # print("----")
      a=str(file).split("/")
      m = a.index("bic_etl")
      b = a[m+1:-1]
      group = b[0]
      ll = splitL(b)
      bdir = "/".join(b)  
      print("GROUP ",group)
      
    #  groups.append(ll)
      for val in data:
            if "title" in val:
                  title=val["title"]
                  info[title]={}
                  info[title]["directory"] = bdir
                  info[title]["group"] = group
                
    #              print(title,ll)
                  nit=0
                  ll = go_deeper(ll,title,nit)
             #     info[title]["groups"] = ll
            
     #             print(ll)
     #             groups.append(ll)
            dataSets.append(val)
    #  print("FF ",ll) 
      groups.append(ll)

    datasets = []
    dataSetsEtl={}
    for val in dataSets:
        if "title" in val:
          datasets.append(val["title"])
          title=val["title"]
          dataSetsEtl[title] = {}  
          dataSetsEtl[title]["info"] = {}
          dataSetsEtl[title]["info"]["directory"] = info[title]["directory"]
          dataSetsEtl[title]["info"]["group"] = info[title]["group"]
     #     dataSetsEtl[title]["info"]["groups"] = info[title]["groups"]
            
        
            
          for k,v in val.items():
          #      print(k,v)
                if k != "title":
                    dataSetsEtl[title][k] = {}
                    if isinstance(v,dict):
                        for k1,v1 in v.items():
                            dataSetsEtl[title][k][k1]=v1

    return sorted(datasets)
init()

In [ ]:
groupD = {}
for gp in groups:
    for k1,v1 in gp.items():
        print(k1,type(v1))
    
        if isinstance(v1,dict):
            for k2,v2 in v1.items():
                print("  K2 ",k2,type(v2))
                if isinstance(v2,dict):
                    for k3,v3 in v2.items():
                        print("   K3  ",k3,type(v3))
                        if k1 not in groupD:
                            groupD[k1]={}
                        if k2 not in groupD[k1]:
                            groupD[k1][k2]={}
                            
                        groupD[k1][k2][k3]=v3            
                        
                        
                else:
                    if k1 not in groupD:
                        groupD[k1] = {}
                    if k2 not in groupD[k1]:
                        groupD[k1][k2] = v2
                    
                            
        else:
            groupD[k1] = v1
            

In [ ]:
groupMenu = []
for k1,v1 in sorted(groupD.items()):
    print(k1)
    if isinstance(v1,list):
        groupMenu.append(k1)
        groupMenu.append(v1)
    else:
        for k2,v2 in v1.items():
            if isinstance(v2,list):
               if k1 not in groupMenu:
                  groupMenu.append(k1)
               if k2 not in groupMenu:
                  groupMenu.append(k2)
               groupMenu.append(v2)
            else:
               for k3,v3 in v2.items():
                   if isinstance(v3,list):
                       if k1 not in groupMenu:
                         groupMenu.append(k1)
                       if k2 not in groupMenu:
                         groupMenu.append(k2)
                        
                       groupMenu.append(k3)                     
                       groupMenu.append(v3)


gm = ["Groups",groupMenu]
gm

In [ ]:
groupMenu = ['Groups',
 ['boulder',
  ['Restaurant Inspections in Boulder Colorado'],
  'catalog',
  ['CIM Catalog Download'],
  'cdhe',
  ['Enrollment Demographics for Post-Secondary Graduates in Colorado',
   'Post-Secondary Financial Aid Demographics in Colorado'],
  'cdor',[
  'revenue_marijuana',
  ['Marijuana Sales by County in Colorado',
   'State Retail Marijuana Sales Tax Revenue by County in Colorado',
   'Marijuana Tax and Fee Revenue in Colorado',
   'Marijuana Sales Revenue in Colorado'],
  'retail_reports',
  ['Retail Sales Tax Return History in Colorado',
   'Retail Reports by City in Colorado',
   'Retail Reports by County in Colorado',
   'Retail Reports by Industry and City in Colorado',
   'Retail Reports by Industry and County in Colorado',
   'Retail Reports by Industry in Colorado'],
  'regulations_liquor',
  ['Liquor Permits for Special Events in Colorado',
   'Liquor Compliance Check Statistics in Colorado',
   'Liquor Licenses in Colorado',
   'Recently Approved Liquor Licenses in Colorado',
   'Recently Expired and Surrendered Liquor Licenses in Colorado',
   'Sales Rooms in Colorado',
   'Manufacturer Temporary Sales Room Permits in Colorado']
  ],
  'cdos',[
  'health',
  ['Durable Medical Equipment Suppliers in Colorado'],
  'government',
  ['Current Notaries in Colorado'],
  'business',[
  'ucc',
  ['Uniform Commercial Code (UCC) Collateral Information in Colorado',
   'Uniform Commercial Code (UCC) Debtor Information in Colorado',
   'Uniform Commercial Code (UCC) Filing Information in Colorado',
   'Secured Party Information in Colorado'],
  'business',
  ['Business Entities in Colorado',
   'Business Entity Transaction History',
   'Trademarks for Businesses in Colorado',
   'Trade Names for Businesses in Colorado',
   'Master List in Colorado'],
  'nonprofit',
  ['Federal Tax-Exempt Subsection Codes in Colorado',
   'Registration for Charities, Paid Solicitors, Professional Fundraising Consultants, and for-profit Public Benefit Corporations in Colorado',
   'Charitable Organizations’ Offices in Colorado',
   'Other State Solicitation of Charities’ Registrants in Colorado',
   'Charitable Purpose of the Charity in Colorado',
   'Paid Solicitor Solicitation Notices in Colorado',
   'Campaign Reports for Solicitation Notices to Charities in Colorado',
   'Solicitation Campaign Supervisors Listed on Solicitation Notices in Colorado',
   'Charity Extension Requests',
   'Persons Associated with Charitable Organizations, Paid Solicitors, and Professional Fundraising Consultants in Colorado',
   'Other Names a Registered Entity Uses to Solicit Contributions',
   'Paid Solicitors Disclosed on Charity Registration Forms in Colorado',
   'Charitable Solicitation Call Center Locations in Colorado',
   'Charities Solicitation Type by Solicitation in Colorado',
   'Communication Methods Used in Solicitation Campaigns in Colorado']
  ],
  'lobbyist',
  ['Directory of Lobbyists in Colorado',
   'Directory of Lobbyist Clients in Colorado',
   'Expenses for Lobbyists in Colorado',
   'Characterization of Lobbyist Clients in Colorado',
   'Subcontractors for Lobbyists in Colorado',
   'Bill Information and Position with Income of Lobbyist in Colorado']
  ],
  'cdot',[
  'transportation_road_attributes',
  ['Highway Milepoints in Colorado',
   'Highway Mileposts in Colorado',
   'Highway Routes in Colorado',
   'Highway Routes in Colorado',
   'Local Roads in Colorado',
   'Major Roads in Colorado',
   'Scenic Byways in Colorado'],
  'tops',
  ['CDOT Expenses', 'CDOT Revenues', 'CDOT Payroll'],
  'natural_resources',
  ['Lakes in Colorado', 'Streams in Colorado'],
  'transportation_infrastructure',
  ['Airports in Colorado',
   'Cities in Colorado',
   'Counties in Colorado',
   'Railroads in Colorado']
  ],
  
  'ceo',[
  'useia',
  ['Gasoline Prices in Colorado', 'Natural Gas Prices in Colorado']
  ],
  'denver',
  ['Temporary Outdoor Expansions for Restaurants in Denver, Colorado'],
  'dola',[
  'special_districts',
  ['Metro Districts in Colorado',
   'Parks and Rec Districts in Colorado',
   'Fire Districts in Colorado',
   'Hospital Districts in Colorado',
   'Water and Sanitation Districts in Colorado',
   'Library Districts in Colorado',
   'School Districts in Colorado',
   'Soil Districts in Colorado',
   'Cemetery Districts in Colorado',
   'All Special Districts in Colorado'],
  'boundaries',
  ['Municipal Annexations in Colorado', 'Municipal Boundaries in Colorado'],
  'demographics',
  ['Population Projections in Colorado',
   'Race Estimates in Colorado',
   'Race Forecast in Colorado']
  ],
  'dora',[
  'regulations',
  ['Licensed Real Estate Professionals in Colorado',
   'Professional and Occupational Licenses in Colorado']
  ],
  'dpa',
  ['DPA Tops Data'],
  'irs',
  ['Purpose and Operational Size of Charities Operating in Colorado',
   'Fundraising Revenue of Charities Operating in Colorado',
   'Total Revenue of Charities Operating in Colorado',
   'IRS Filing Information for Charities Operating in Colorado',
   'Total Revenue and Types of Art for Charities Operating in Colorado',
   'Conservation Easements for Charities Operating in Colorado',
   'Activities of Charities Operating in Colorado',
   'Expenses of Charities Operating in Colorado',
   'Expenses of Charities Operating in Colorado'],
  'tchd',
  ['Restaurant Inspections in Tri-County Colorado']]]

In [ ]:
import PySimpleGUI as sg

# menu_def = ['Location',
#     [
#         'Rack 1',
#             [
#                 'Shelf 1', ['Bin A::11', 'Bin B::11'],
#                 'Shelf 2', ['Bin A::12', 'Bin B::12'],
#             ],
#         'Rack 2',
#             [
#                 'Shelf 1', ['Bin A::21', 'Bin B::21'],
#                 'Shelf 2', ['Bin A::22', 'Bin B::22'],
#             ],
#     ]
# ]
menu_def = gm
layout = [[sg.ButtonMenu('Location', menu_def=menu_def, key='Button_Menu')],
          [sg.Button("Close")]]
window = sg.Window('Title', layout)

while True:
    event, values = window.read()
    print(event,values)
    if event == sg.WIN_CLOSED or event == "Close":
        break
    elif event == 'Button_Menu':
        print(values[event])

window.close()

In [ ]:
def fullDataColumnMap(cim4x4):
    global missed
    if cim4x4 in fields:
        invc = fields[cim4x4]["source"].copy()
    else:
        invc = []
    tfc = transformColumns.copy()
    exc = extractColumns.copy()
    excMissed = []
    tfcsij = transformSijColumns.copy()
    loadsij = loadSijColumns.copy()
    cim = cimColumns.copy()
    xrfo2t = xrefsO2T.copy()
    names = []
    columnDict = {}
    names.append("BIC Inventory")
    names.append("Transform Program")

    names.append("Transform - Data")
    names.append("CIM - Data")
    names.append("Transform - SIJ")
    names.append("Load - SIJ")

    for k in exc:
        print("MAP ",k)
    #for k,v in xrefsO2T.items():
        columnDict[k] = []
        nhit=0
        v=""
        if k in invc:
            v=k
            vv=v
            invc.remove(k)
            nhit+=1
        else:
            vv=""
        columnDict[k].append(vv)
        
        if k in xrfo2t:
            v = xrfo2t[k]
            vv=v
            del xrfo2t[k]
            nhit+=1
        else:
            vv = ""
        columnDict[k].append(vv)  

   
        if v in tfc:
           tfc.remove(v)
           cc = v
           nhit+=1
        else:
            cc=""
        columnDict[k].append(cc)


        if v in cim:
           cim.remove(v)
           cc = v
           nhit+=1            
        else:
            cc=""
        columnDict[k].append(cc)



        if v.lower() in tfcsij:
           tfcsij.remove(v.lower())
           cc = v.lower()
           nhit+=1            
        else:
            cc=""
        columnDict[k].append(cc)

        if v.lower() in loadsij:
           loadsij.remove(v.lower())
           cc = v.lower()
           nhit+=1
        else:
            cc=""
        columnDict[k].append(cc)
        
        if nhit == 0:
            excMissed.append(k)
        

    a = pd.DataFrame(columnDict).T
    a.columns = names
    a.reset_index(inplace=True)
    values = a.values.tolist()
    cols = list(a.columns)
   
    missed = {}
    missed["Extract - Data"] = excMissed
    missed["BIC Inventory"] = invc
    missed["Transform Program"] = list(xrfo2t.keys())
    missed["Transform - Data"] = tfc
    missed["CIM - Data"] = cim
    missed["Transform - SIJ"] = tfcsij
    missed["Load - SIJ"] = loadsij
    
    
 #   missed = [invc,list(xrfo2t.keys()),tfc,cim,tfcsij,loadsij]
    return cols,values,missed
        
a,b,c = fullDataColumnMap("w6kb-3vsj")

In [ ]:
xrefsO2T